In [ ]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

In [ ]:
# 1) Write a Python function that builds and trains the model, and outputs the
# final predictions and losses

def build_model():
    return nn.Sequential(
        nn.Linear(1,1),  # input layer
        nn.ReLU(),       # activation function
        nn.Linear(1,1)   # output layer
    )

def train_model(x, y, loss_function=nn.MSELoss(), num_epochs=500):
    ANNreg = build_model()
    optimizer = torch.optim.SGD(ANNreg.parameters(), lr=.05)
    losses = torch.zeros(num_epochs)

    ## Train the model
    for epochi in range(num_epochs):
        # forward pass
        yHat = ANNreg(x)
        # compute loss
        loss = loss_function(yHat,y)
        losses[epochi] = loss
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # manually compute losses
    # final forward pass
    predictions = ANNreg(x)
    
    return predictions, losses

In [ ]:
# 2) Write a function that creates the data and outputs x, y
# y = m * x + randn / 2
# N = 50

def create_data(m, N=50):
    x = torch.randn(N,1)
    y = m * x + torch.randn(N,1)/2
    return x, y

In [ ]:
# 3) Parametric experiment
# Vary the slope from -2 to 2 in 21 steps
# Repeat the entire experiment 50 times

def create_slopes_in_n_steps(init_m, final_m, steps):
    return np.linspace(init_m, final_m, steps)

slopes = create_slopes_in_n_steps(-2, 2, 21)
# print("test slopes", len(slopes), slopes)

# RUN THE EXPERIMENT
number_experiments = 50
# initialize output matrix (taken from result)
results = np.zeros((len(slopes), number_experiments,2))

for slope_idx in range(len(slopes)):
    print(f"slope = {slopes[slope_idx]}")
    for experiment in range(number_experiments):
        # print(f"experiment = {experiment + 1}")
        x, y = create_data(slopes[slope_idx])
        y_hat, losses = train_model(x, y)
        y_hat = y_hat.detach()

        # store the final loss and performance (taken from result)
        results[slope_idx, experiment, 0] = losses[-1]
        results[slope_idx, experiment, 1] = np.corrcoef(y.T, y_hat.T)[0, 1]

# correlation can be 0 if the model didn't do well. Set nan's->0
# (taken from result)
results[np.isnan(results)] = 0


In [ ]:
# 4) Plot the loss and accuracy (correlation of y_hat, y) as a function of the
# slope

fig, ax = plt.subplots(1, 2, figsize=(12,4))

ax[0].plot(slopes, np.mean(results[:, :, 0],axis=1), 'ko-', markerfacecolor='w', markersize=10)
ax[0].set_xlabel('Slope')
ax[0].set_title('Loss')

ax[1].plot(slopes, np.mean(results[:, :, 1],axis=1), 'ms-', markerfacecolor='w', markersize=10)
ax[1].set_xlabel('Slope')
ax[1].set_ylabel('Real-predicted correlation')
ax[1].set_title('Model performance')

plt.show()

In [ ]:
# extra code to visualize data with different slopes

m = 2

x,y = create_data(m)

plt.title('Slope = ' + str(m))
plt.plot(x,y,'o')
plt.ylim([-4,4])
plt.xlabel('x')
plt.ylabel('y')
plt.show()